https://www.topozone.com/oregon/multnomah-or/reservoir/city-of-portland-reservoir-number-3/

# The Little Pascals : Project 1 Calculator

Here I want to add an explanation of our process, what equations used, and why

We were given the diameters, lengths, types of bends per length, and their materials for the piping sections.
so to determine the head for the pump for each reservoir from the equation:

$\frac{P_{1}}{\rho g} + \frac{V_{1}^2}{2g} +z_{1} =  \frac{P_{2}}{\rho g} + \frac{V_{2}^2}{2g} +z_{2} + h_{p} - h_{maj} - h_{min}$

Rearranged to (as the minor losses would be incorporated through equivalent lengths): 

$h_{p} = (\frac{P_{1}}{\rho g} + \frac{V_{1}^2}{2g} +z_{1}) - (\frac{P_{2}}{\rho g} + \frac{V_{2}^2}{2g} +z_{2} ) + h_{maj}$

So we apply this process to find head from the pump:

1. Determine elevations of OHSU and the Reservoirs
2. Find their hydraulic diameters for non circular pipes:
    
    $D_{h} = \frac{4A_{h}}{P}$
    
3. Find the areas of all the pipes with diameters
4. Find the flow rate from the maximum volume/month
5. Determine velocities using:

    $V = \frac{Q}{A}$
    
    
6. Find the Reynolds Number:

   $ Re = \frac{VD}{\nu}$
   
   
7. Convert the types of bends and materials to k and roughness values (with one array with concrete switched out with steel)

8. Find k values between changes of diameter in piping with:

    $k = (1 - \frac{A_{1}}{A_{2}})^2$


9. Determine friciton factors from :

    $\textit{f} = \frac{64}{Re} = \frac{1}{-1.8*log_{10}((\frac{6.9}{Re})+(\frac{\epsilon/d}{3.7})^{1.11}))}$
    
    
10. Determine equivalent lengths from:

    $Le = \frac{kD}{f}$
    
    
11. Find the head loss from each section with: 

    $h_{maj} = \frac{fL_eV^2}{2gD}$
    
    
12. For the other variables in the head for head in the pump:
    Pressure at Reservoir = 0 (gage)
    Pressure at OHSU = 80 psi (average underground water pipe pressure)
    Velocities from steps above.
    Assume water is 60 degrees F
13. Plug in!
14. Use head to convert to pressure with:

    $P = \frac{h_{p}}{2.31}$
    
    
15. Use this relationship to solve for Energy (we have desired pressure and volume)
    
    $Pressure = \frac{Energy}{Unit Volume}$
    
    
16. Convert energy to kWh 
17. Find cost of energy and divide by 526 beds to find cost per bed.
18. Determine from various costs which one would be the best.

## Reservoir 3 Calculations:

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
from sympy import symbols, solve 
OHSUelv=450
Res3elv=295
#Read excel file + Pick out reservoir section by title and relevant data
df = pd.read_csv("data/pipes.csv")
res3= df[df["Reservoir"] == "City of Portland Reservoir 3"]  
res3 = res3[["Lengths [ft]","Diameter [ft]","Components","Material"]]  

#Seperate lists of info (component and diameter sections to individual columns)
res3['Components'] = res3['Components'].map(lambda x: x.strip("['']''"))
res3['Diameter [ft]'] = res3['Diameter [ft]'].map(lambda x: x.strip("()"))
res3[['d1','d2']] = res3['Diameter [ft]'].str.split(",",expand=True)
res3[['c1','c2','c3','c4','c5']] = res3['Components'].str.split("', '",expand=True)

#Swap out different bend types for their k values (table 8.2) and roughness values for e (table 8.1)
mapping={"Globe Valve (Open)":10,"Elbow 90 deg flanged":0.3, 
          "180 Bend (Flanged)":0.2,"Union (Threaded)":0.08,
         "Gate (Open)":0.15,"Elbow 90 deg threaded":1.5,"Ball Valve (Open)":0.05,
         "180 bend (Threaded)":1.5,"Long Radius 90 deg threaded":0.7,
         "Long Radius 90 deg flanged":0.2,"Swing Check (forward flow)":2,
         "Long radius 45 deg flanged":0.2,"45 deg threaded":0.4,
         "Ball Valve (Half Open)":107,"Long radius 45 deg threaded":.7,"":0,
         "Concrete":.001,"Iron":.00085,"Commerical Steel":.00015,"Old Concrete":.01,
         "Galvanized Iron":.0005,
}
res3 = res3.replace({'c1':mapping,'c2':mapping,'c3':mapping,'c4':mapping,'c5':mapping,'Material':mapping})

#Sum their k values for each section of piping
kval = res3[["c1","c2","c3","c4","c5"]].sum(1)

#Finding the areas of each piping sections
# where d2 is 0 if piping circular
dia1 = res3["d1"] #pulled the columns out of our pandas
dia2 = res3["d2"]
d1 = dia1.to_numpy() #converted each d1 column into a numpy array
d2 = dia2.to_numpy()

def area(d1, d2):
    if (d2 != None): 
        d1 = float(d1) #turned the strings into float (a float is a decimal number)
        d2 = float(d2) 
        A = d1*d2
        
    elif (d2 == None):
        d1 = float(d1)
        A = (np.pi/4)*d1**2
        
    return A
Area_array = []

for i in range(len(d1)): #to make the function go through the array 
    Area_array.append(area(d1[i], d2[i])) 

#Finding the velocity in each section using the maximum volume/month in invoices & Q = VA
aa = np.array(Area_array)
dt=pd.read_csv('data/invoices.csv')
vol=dt["Water Volume [ccf]"]
Qmonth=max(vol)*100
Qday=Qmonth/30
Qhour=Qday/24
Qminute=Qhour/60
Q=Qminute/60

for i in range(len(aa)):
    V=Q/aa

#Hydraulic Diameter for non-circular sections & adding it back into to the array of diameters
dia1 = res3["d1"] #pulled the columns out of our pandas
dia2 = res3["d2"]
r1 = dia1.to_numpy() #converted each d1 column into a numpy array
r2 = dia2.to_numpy()

def D_hyd(r1,r2):
    if(r2 != None):
        r1 = float(r1)
        r2 = float(r2)
        P = 2*r1+2*r2
        Ah = r1*r2
        D_h = 4*Ah/P
    elif(r2 == None):
        r1=float(r1)
        D_h = r1
    return D_h
D_array = []

for s in range(len(d1)):
    D_array.append(D_hyd(d1[s],d2[s])) 
    
#Reynolds numbers
dh = np.array(D_array)
nu = 1.210*10**-5 #kinematic viscocity of water at 60 deg F
Re = (dh*V)/nu

for i in range(len(dh)):
    Re = (dh*V)/nu

#Friction Factors
# define roughness values to the material column
m=res3["Material"]
e = np.array(m)

def fric_fac(Re,e,dh):
    if (Re>2000):
        f=(1/(-1.8*np.log10((6.9/Re)+(((e/dh)/3.7)**1.11))))**2 #friction factor equation 
    elif (Re<2000):
        f=64/Re
    return f
fric_array = []
    
for i in range(len(dh)):
    fric_array.append(fric_fac(Re[i],e[i],dh[i]))
f_array = np.array(fric_array)    


#Works out the expansions and gives us their K values
Area1=np.array(Area_array)         #This changes the list Area_array to a array
Ar1=pd.DataFrame(Area1)
A1=Ar1[:-1]#This makes a dataframe out of the previous array 
A2=Ar1[1:]                       #This tells the code that A2 is the same as A1 but starting from the 2nd position
a1=np.array(A1)                  #These change us back to arrays now that we have redefined the dataframes
a2=np.array(A2)
check=np.greater(a1,a2)
def Exp (a1,a2,check):
    if (check==True):
        Exa=(1-(a1/a2))**2 #This is just the expansion coeffiecient equation 
    elif (check==False):
        Exa=(1-(a2/a1))**2
  
    return Exa
K=[]
for i in range(len(a2)):
       K.append(Exp(a1[i],a2[i],check[i]))
k1=np.array(K)

#Equivalent lengths of expansion
for i in range(len(dh)):
    L_ee = k1*dh/f_array
    
#Equivalent lengths of bends/components
kvala = np.array(kval)
for i in range(len(dh)):
    L_eb = kvala*dh/f_array
    
#Headloss of expansions
g = 32.2
h_le = (f_array*L_ee*V**2)/(2*g*dh)
sum_h_le = np.sum(h_le)

#Headloss of bends
h_lb = (f_array*L_eb*V**2)/(2*g*dh)
sum_h_lb = np.sum(h_lb)

#Headloss of lengths
lengths = res3["Lengths [ft]"]
leng = np.array(lengths)
h_ll = (f_array*leng*V**2)/(2*g*dh)
sum_h_ll = np.sum(h_ll)

#Total head loss
h_ltot = sum_h_ll + sum_h_lb + sum_h_le


#solves for pump pressure
hp=symbols('hp')
solver=Res3elv+hp-h_ltot-11520-OHSUelv
hp=solve(solver)
Hp=np.array(hp)
pr=Hp/2.31

#solves for pump energy
W=pr*Q*24/1000
W  #This is Kilowatt hours per day 

# Cost of Energy (average for businesses)
energycost = 0.127
cost = energycost*W  #this is cost per day 


# Cost per bed
cost_bed = cost/526


#Install cost
In=235000/365

#Lease

Lea=25000/365

#Insurance

Ins=3800/365

#Cost per day
Cost=cost+In+Lea+Ins
print('Total cost per day is $', Cost,'and the cost per bed per day is $', cost_bed)


Total cost per day is $ [723.754054243000] and the cost per bed per day is $ [0.00192838063802823]


## Renovated Reservior 3 Calculations

In [2]:
##Same as above but converting the concrete roughness to that of commercial steel 

dt= pd.read_csv("data/pipes.csv")
res3ren=dt[dt["Reservoir"] == "City of Portland Reservoir 3"]  
res3ren = res3ren[["Material","Lengths [ft]"]]
mappingren={"Concrete":.00015,"Iron":.00085,"Commerical Steel":.00015,"Old Concrete":.00015,
         "Galvanized Iron":.0005,}
res3ren = res3ren.replace({'Material':mappingren})
#Friction Factors
# define roughness values to the material column
mren=res3ren["Material"]
eren = np.array(mren)

def renfric_fac(Re,eren,dh):
    if (Re>2000):
        f=(1/(-1.8*np.log10((6.9/Re)+(((eren/dh)/3.7)**1.11))))**2 #friction factor equation 
    elif (Re<2000):
        f=64/Re
    return f
renfric_array = []
    
for i in range(len(dh)):
    renfric_array.append(renfric_fac(Re[i],eren[i],dh[i]))
renf_array = np.array(renfric_array)

#Equivalent lengths of expansion
for i in range(len(dh)):
    renL_ee = k1*dh/renf_array
    
#Equivalent lengths of bends/components
kvala = np.array(kval)
for i in range(len(dh)):
    renL_eb = kvala*dh/renf_array
    
#Headloss of expansions
g = 32.2
renh_le = (renf_array*renL_ee*V**2)/(2*g*dh)
rensum_h_le = np.sum(renh_le)

#Headloss of bends
renh_lb = (renf_array*renL_eb*V**2)/(2*g*dh)
rensum_h_lb = np.sum(renh_lb)

#Headloss of lengths
lengths = res3ren["Lengths [ft]"]
leng = np.array(lengths)
renh_ll = (renf_array*leng*V**2)/(2*g*dh)
rensum_h_ll = np.sum(renh_ll)

#Total head loss
renh_ltot = rensum_h_ll + rensum_h_lb + rensum_h_le


#solves for pump pressure
hp=symbols('hp')
solver=Res3elv+hp-renh_ltot-11520-OHSUelv
hp=solve(solver)
hp


#solves for pump pressure
hp=symbols('hp')
solver=Res3elv+hp-renh_ltot-11520-OHSUelv
hp=solve(solver)
Hp=np.array(hp)
pr=Hp/2.31

#Solves for pump energy
W=pr*Q*24/1000
W

# Cost of Energy (average for businesses)
energycost = 0.127
cost = energycost*W


# Cost per bed
cost_bed = cost/526


#Install cost
In=235000/365

#Lease

Lea=25000/365

#Insurance

Ins=3800/365

#Cost per day
Cost=cost+In+Lea+Ins
print('Total cost per day is $', Cost,'and the cost per bed per day is $', cost_bed)

Total cost per day is $ [723.754054011663] and the cost per bed per day is $ [0.00192838019822421]


## Reservoir 4 Calculations

In [3]:
#Import Libraries
import pandas as pd
import numpy as np
from sympy import symbols, solve
Res4elv=226

#Read excel file + Pick out reservoir section by title and relevant data
df = pd.read_csv("data/pipes.csv")
res4= df[df["Reservoir"] == "City of Portland Reservoir 4"]  
res4 = res4[["Lengths [ft]","Diameter [ft]","Components","Material"]]  

#Seperate lists of info (component and diameter sections to individual columns)
res4['Components'] = res4['Components'].map(lambda x: x.strip("['']''"))
res4['Diameter [ft]'] = res4['Diameter [ft]'].map(lambda x: x.strip("()"))
res4[['d1','d2']] = res4['Diameter [ft]'].str.split(",",expand=True)
res4[['c1','c2','c3','c4','c5']] = res4['Components'].str.split("', '",expand=True)

#Swap out different bend types for their k values (table 8.2) and roughness values for e (table 8.1)
mapping={"Globe Valve (Open)":10,"Elbow 90 deg flanged":0.3, 
          "180 Bend (Flanged)":0.2,"Union (Threaded)":0.08,
         "Gate (Open)":0.15,"Elbow 90 deg threaded":1.5,"Ball Valve (Open)":0.05,
         "180 bend (Threaded)":1.5,"Long Radius 90 deg threaded":0.7,
         "Long Radius 90 deg flanged":0.2,"Swing Check (forward flow)":2,
         "Long radius 45 deg flanged":0.2,"45 deg threaded":0.4,
         "Ball Valve (Half Open)":107,"Long radius 45 deg threaded":.7,"":0,
         "Concrete":.001,"Iron":.00085,"Commerical Steel":.00015,"Old Concrete":.01,
         "Galvanized Iron":.0005,
}
res4 = res4.replace({'c1':mapping,'c2':mapping,'c3':mapping,'c4':mapping,'c5':mapping,'Material':mapping})

#Sum their k values for each section of piping
kval = res4[["c1","c2","c3","c4","c5"]].sum(1)

#Finding the areas of each piping sections
# where d2 is 0 if piping circular
dia1 = res4["d1"] #pulled the columns out of our pandas
dia2 = res4["d2"]
d1 = dia1.to_numpy() #converted each d1 column into a numpy array
d2 = dia2.to_numpy()

def area(d1, d2):
    if (d2 != None): 
        d1 = float(d1) #turned the strings into float (a float is a decimal number)
        d2 = float(d2) 
        A = d1*d2
        
    elif (d2 == None):
        d1 = float(d1)
        A = (np.pi/4)*d1**2
        
    return A
Area_array = []

for i in range(len(d1)): #to make the function go through the array 
    Area_array.append(area(d1[i], d2[i])) 

#Finding the velocity in each section using the maximum volume/month in invoices & Q = VA
aa = np.array(Area_array)
dt=pd.read_csv('data/invoices.csv')
vol=dt["Water Volume [ccf]"]
Qmonth=max(vol)*100
Qday=Qmonth/30
Qhour=Qday/24
Qminute=Qhour/60
Q=Qminute/60

for i in range(len(aa)):
    V=Q/aa

#Hydraulic Diameter for non-circular sections & adding it back into to the array of diameters
dia1 = res4["d1"] #pulled the columns out of our pandas
dia2 = res4["d2"]
r1 = dia1.to_numpy() #converted each d1 column into a numpy array
r2 = dia2.to_numpy()

def D_hyd(r1,r2):
    if(r2 != None):
        r1 = float(r1)
        r2 = float(r2)
        P = 2*r1+2*r2
        Ah = r1*r2
        D_h = 4*Ah/P
    elif(r2 == None):
        r1=float(r1)
        D_h = r1
    return D_h
D_array = []

for s in range(len(d1)):
    D_array.append(D_hyd(d1[s],d2[s])) 
    
#Reynolds numbers
dh = np.array(D_array)
nu = 1.210*10**-5 #kinematic viscocity of water at 60 deg F
Re4 = (dh*V)/nu

for i in range(len(dh)):
    Re4 = (dh*V)/nu

#Friction Factors
# define roughness values to the material column
m=res4["Material"]
e = np.array(m)

def fric_fac(Re4,e,dh):
    if (Re4>2000):
        f=(1/(-1.8*np.log10((6.9/Re4)+(((e/dh)/3.7)**1.11))))**2 #friction factor equation 
    elif (Re4<2000):
        f=64/Re4
    return f
fric_array = []
    
for i in range(len(dh)):
    fric_array.append(fric_fac(Re[i],e[i],dh[i]))
f_array = np.array(fric_array)    


#Works out the expansions and gives us their K values
Area1=np.array(Area_array)         #This changes the list Area_array to a array
Ar1=pd.DataFrame(Area1)
A1=Ar1[:-1]#This makes a dataframe out of the previous array 
A2=Ar1[1:]                       #This tells the code that A2 is the same as A1 but starting from the 2nd position
a1=np.array(A1)                  #These change us back to arrays now that we have redefined the dataframes
a2=np.array(A2)
check=np.greater(a1,a2)
def Exp (a1,a2,check):
    if (check==True):
        Exa=(1-(a1/a2))**2 #This is just the expansion coeffiecient equation 
    elif (check==False):
        Exa=(1-(a2/a1))**2
  
    return Exa
    K=[]
for i in range(len(a2)):
       K.append(Exp(a1[i],a2[i],check[i]))
k1=np.array(K)

#Equivalent lengths of expansion
for i in range(len(dh)):
    L_ee = k1*dh/f_array
    
#Equivalent lengths of bends/components
kvala = np.array(kval)
for i in range(len(dh)):
    L_eb = kvala*dh/f_array
    
#Headloss of expansions
g = 32.2
h_le = (f_array*L_ee*V**2)/(2*g*dh)
sum_h_le = np.sum(h_le)

#Headloss of bends
h_lb = (f_array*L_eb*V**2)/(2*g*dh)
sum_h_lb = np.sum(h_lb)

#Headloss of lengths
lengths = res4["Lengths [ft]"]
leng = np.array(lengths)
h_ll = (f_array*leng*V**2)/(2*g*dh)
sum_h_ll = np.sum(h_ll)

#Total head loss
h_ltot = sum_h_ll + sum_h_lb + sum_h_le


#solves for pump pressure
hp=symbols('hp')
solver=Res4elv+hp-h_ltot-11520-OHSUelv
hp=solve(solver)
hp


#solves for pump pressure
hp=symbols('hp')
solver=Res3elv+hp-h_ltot-11520-OHSUelv
hp=solve(solver)
Hp=np.array(hp)
pr=Hp/2.31
W=pr*Q*24/1000
W

# Cost of Energy (average for businesses)
energycost = 0.127
cost = energycost*W


# Cost per bed
cost_bed = cost/526


#Install cost
In=185000/365

#Lease

Lea=32000/365

#Insurance

Ins=5200/365

#Cost per day
Cost=cost+In+Lea+Ins
print('Total cost per day is $', Cost,'and the cost per bed per day is $', cost_bed)

Total cost per day is $ [609.783668145482] and the cost per bed per day is $ [0.00193259478671335]


## Renovated Reservoir 4 

In [4]:
dt= pd.read_csv("data/pipes.csv")
res4ren=dt[dt["Reservoir"] == "City of Portland Reservoir 4"]  
res4ren = res4ren[["Material","Lengths [ft]"]]
mappingren={"Concrete":.00015,"Iron":.00085,"Commerical Steel":.00015,"Old Concrete":.00015,
         "Galvanized Iron":.0005,}
res4ren = res4ren.replace({'Material':mappingren})
#Friction Factors
# define roughness values to the material column
mren=res4ren["Material"]
eren = np.array(mren)

def renfric_fac(Re4,eren,dh):
    if (Re4>2000):
        f=(1/(-1.8*np.log10((6.9/Re4)+(((eren/dh)/3.7)**1.11))))**2 #friction factor equation 
    elif (Re4<2000):
        f=64/Re4
    return f
renfric_array = []
    
for i in range(len(dh)):
    renfric_array.append(renfric_fac(Re4[i],eren[i],dh[i]))
renf_array = np.array(renfric_array)

#Equivalent lengths of expansion
for i in range(len(dh)):
    renL_ee = k1*dh/renf_array
    
#Equivalent lengths of bends/components
kvala = np.array(kval)
for i in range(len(dh)):
    renL_eb = kvala*dh/renf_array
    
#Headloss of expansions
g = 32.2
renh_le = (renf_array*renL_ee*V**2)/(2*g*dh)
rensum_h_le = np.sum(renh_le)

#Headloss of bends
renh_lb = (renf_array*renL_eb*V**2)/(2*g*dh)
rensum_h_lb = np.sum(renh_lb)

#Headloss of lengths
lengths = res4ren["Lengths [ft]"]
leng = np.array(lengths)
renh_ll = (renf_array*leng*V**2)/(2*g*dh)
rensum_h_ll = np.sum(renh_ll)

#Total head loss
renh_ltot = rensum_h_ll + rensum_h_lb + rensum_h_le

#solves for pump pressure
hp=symbols('hp')
solver=Res4elv+hp-renh_ltot-11520-OHSUelv
hp=solve(solver)


#solves for pump pressure
hp=symbols('hp')
solver=Res4elv+hp-renh_ltot-11520-OHSUelv
hp=solve(solver)
Hp=np.array(hp)
pr=Hp/2.31

#solves for pump energy
W=pr*Q*24/1000
W

# Cost of Energy (average for businesses)
energycost = 0.127
cost = energycost*W


# Cost per bed
cost_bed = cost/526

#Install cost
In=185000/365

#Lease

Lea=32000/365

#Insurance

Ins=5200/365

#Cost per day
Cost=cost+In+Lea+Ins
print('Total cost per day is $', Cost,'and the cost per bed per day is $', cost_bed)

Total cost per day is $ [609.789498027480] and the cost per bed per day is $ [0.00194367821256422]


## Cemetery Reservoir

In [5]:
#Import Libraries
import pandas as pd
import numpy as np
Cemelv=525
#Read excel file + Pick out reservoir section by title and relevant data
df = pd.read_csv("data/pipes.csv")
cem= df[df["Reservoir"] == "Cemetery Reservoir"]  
cem = cem[["Lengths [ft]","Diameter [ft]","Components","Material"]]  

#Seperate lists of info (component and diameter sections to individual columns)
cem['Components'] = cem['Components'].map(lambda x: x.strip("['']''"))
cem['Diameter [ft]'] = cem['Diameter [ft]'].map(lambda x: x.strip("()"))
cem[['d1','d2']] = cem['Diameter [ft]'].str.split(",",expand=True)
cem[['c1','c2','c3','c4','c5']] = cem['Components'].str.split("', '",expand=True)

#Swap out different bend types for their k values (table 8.2) and roughness values for e (table 8.1)
mapping={"Globe Valve (Open)":10,"Elbow 90 deg flanged":0.3, 
          "180 Bend (Flanged)":0.2,"Union (Threaded)":0.08,
         "Gate (Open)":0.15,"Elbow 90 deg threaded":1.5,"Ball Valve (Open)":0.05,
         "180 bend (Threaded)":1.5,"Long Radius 90 deg threaded":0.7,
         "Long Radius 90 deg flanged":0.2,"Swing Check (forward flow)":2,
         "Long radius 45 deg flanged":0.2,"45 deg threaded":0.4,
         "Ball Valve (Half Open)":107,"Long radius 45 deg threaded":.7,"":0,
         "Concrete":.001,"Iron":.00085,"Commerical Steel":.00015,"Old Concrete":.01,
         "Galvanized Iron":.0005,
}
cem = cem.replace({'c1':mapping,'c2':mapping,'c3':mapping,'c4':mapping,'c5':mapping,'Material':mapping})

#Sum their k values for each section of piping
kval = cem[["c1","c2","c3","c4","c5"]].sum(1)

#Finding the areas of each piping sections
# where d2 is 0 if piping circular
dia1 = cem["d1"] #pulled the columns out of our pandas
dia2 = cem["d2"]
d1 = dia1.to_numpy() #converted each d1 column into a numpy array
d2 = dia2.to_numpy()

def area(d1, d2):
    if (d2 != None): 
        d1 = float(d1) #turned the strings into float (a float is a decimal number)
        d2 = float(d2) 
        A = d1*d2
        
    elif (d2 == None):
        d1 = float(d1)
        A = (np.pi/4)*d1**2
        
    return A
Area_array = []

for i in range(len(d1)): #to make the function go through the array 
    Area_array.append(area(d1[i], d2[i])) 

#Finding the velocity in each section using the maximum volume/month in invoices & Q = VA
aa = np.array(Area_array)
dt=pd.read_csv('data/invoices.csv')
vol=dt["Water Volume [ccf]"]
Qmonth=max(vol)*100
Qday=Qmonth/30
Qhour=Qday/24
Qminute=Qhour/60
Q=Qminute/60

for i in range(len(aa)):
    V=Q/aa

#Hydraulic Diameter for non-circular sections & adding it back into to the array of diameters
dia1 = cem["d1"] #pulled the columns out of our pandas
dia2 = cem["d2"]
r1 = dia1.to_numpy() #converted each d1 column into a numpy array
r2 = dia2.to_numpy()

def D_hyd(r1,r2):
    if(r2 != None):
        r1 = float(r1)
        r2 = float(r2)
        P = 2*r1+2*r2
        Ah = r1*r2
        D_h = 4*Ah/P
    elif(r2 == None):
        r1=float(r1)
        D_h = r1
    return D_h
D_array = []

for s in range(len(d1)):
    D_array.append(D_hyd(d1[s],d2[s])) 
    
#Reynolds numbers
dh = np.array(D_array)
nu = 1.210*10**-5 #kinematic viscocity of water at 60 deg F
Re = (dh*V)/nu

for i in range(len(dh)):
    Re = (dh*V)/nu

#Friction Factors
# define roughness values to the material column
m=cem["Material"]
e = np.array(m)

def fric_fac(Re,e,dh):
    if (Re>2000):
        f=(1/(-1.8*np.log10((6.9/Re)+(((e/dh)/3.7)**1.11))))**2 #friction factor equation 
    elif (Re<2000):
        f=64/Re
    return f
fric_array = []
    
for i in range(len(dh)):
    fric_array.append(fric_fac(Re[i],e[i],dh[i]))
f_array = np.array(fric_array)    


#Works out the expansions and gives us their K values
Area1=np.array(Area_array)         #This changes the list Area_array to a array
Ar1=pd.DataFrame(Area1)
A1=Ar1[:-1]#This makes a dataframe out of the previous array 
A2=Ar1[1:]                       #This tells the code that A2 is the same as A1 but starting from the 2nd position
a1=np.array(A1)                  #These change us back to arrays now that we have redefined the dataframes
a2=np.array(A2)
check=np.greater(a1,a2)
def Exp (a1,a2,check):
    if (check==True):
        Exa=(1-(a1/a2))**2 #This is just the expansion coeffiecient equation 
    elif (check==False):
        Exa=(1-(a2/a1))**2
  
    return Exa
    K=[]
for i in range(len(a2)):
       K.append(Exp(a1[i],a2[i],check[i]))
k1=np.array(K)

#Equivalent lengths of expansion
for i in range(len(dh)):
    L_ee = k1*dh/f_array
    
#Equivalent lengths of bends/components
kvala = np.array(kval)
for i in range(len(dh)):
    L_eb = kvala*dh/f_array
    
#Headloss of expansions
g = 32.2
h_le = (f_array*L_ee*V**2)/(2*g*dh)
sum_h_le = np.sum(h_le)

#Headloss of bends
h_lb = (f_array*L_eb*V**2)/(2*g*dh)
sum_h_lb = np.sum(h_lb)

#Headloss of lengths
lengths = cem["Lengths [ft]"]
leng = np.array(lengths)
h_ll = (f_array*leng*V**2)/(2*g*dh)
sum_h_ll = np.sum(h_ll)

#Total head loss
h_ltot = sum_h_ll + sum_h_lb + sum_h_le


#solves for pump pressure
hp=symbols('hp')
solver=Cemelv+hp-h_ltot-11520-OHSUelv
hp=solve(solver)


#solves for pump pressure
hp=symbols('hp')
solver=Cemelv+hp-h_ltot-11520-OHSUelv
hp=solve(solver)
Hp=np.array(hp)
pr=Hp/2.31
W=pr*Q*24/1000
W

# Cost of Energy (average for businesses)
energycost = 0.127
cost = energycost*W


# Cost per bed
cost_bed = cost/526

#Install cost
In=85000/365

#Lease

Lea=8000/365

#Insurance

Ins=9120/365

#Cost per day
Cost=cost+In+Lea+Ins
print('Total cost per day is $', Cost,'and the cost per bed per day is $', cost_bed)

Total cost per day is $ [281.365281756363] and the cost per bed per day is $ [0.00301228106189193]


## Renovated Cemetery

In [6]:
dt= pd.read_csv("data/pipes.csv")
cemren=dt[dt["Reservoir"] == "Cemetery Reservoir"]  
cemren = cemren[["Material","Lengths [ft]"]]
mappingren={"Concrete":.00015,"Iron":.00085,"Commerical Steel":.00015,"Old Concrete":.00015,
         "Galvanized Iron":.0005,}
cemren = cemren.replace({'Material':mappingren})
#Friction Factors
# define roughness values to the material column
mren=cemren["Material"]
eren = np.array(mren)

def renfric_fac(Re,eren,dh):
    if (Re>2000):
        f=(1/(-1.8*np.log10((6.9/Re)+(((eren/dh)/3.7)*1.11))))**2 #friction factor equation 
    elif (Re<2000):
        f=64/Re
    return f
renfric_array = []
    
for i in range(len(dh)):
    renfric_array.append(renfric_fac(Re[i],eren[i],dh[i]))
renf_array = np.array(renfric_array)

#Equivalent lengths of expansion
for i in range(len(dh)):
    renL_ee = k1*dh/renf_array
    
#Equivalent lengths of bends/components
kvala = np.array(kval)
for i in range(len(dh)):
    renL_eb = kvala*dh/renf_array
    
#Headloss of expansions
g = 32.2
renh_le = (renf_array*renL_ee*V**2)/(2*g*dh)
rensum_h_le = np.sum(renh_le)

#Headloss of bends
renh_lb = (renf_array*renL_eb*V**2)/(2*g*dh)
rensum_h_lb = np.sum(renh_lb)

#Headloss of lengths
lengths = cemren["Lengths [ft]"]
leng = np.array(lengths)
renh_ll = (renf_array*leng*V**2)/(2*g*dh)
rensum_h_ll = np.sum(renh_ll)

#Total head loss
renh_ltot = rensum_h_ll + rensum_h_lb + rensum_h_le

#solves for pump pressure
hp=symbols('hp')
solver=Cemelv+hp-renh_ltot-11520-OHSUelv
hp=solve(solver)


#solves for pump pressure
hp=symbols('hp')
solver=Cemelv+hp-renh_ltot-11520-OHSUelv
hp=solve(solver)
Hp=np.array(hp)
pr=Hp/2.31
W=pr*Q*24/1000

# Cost of Energy (average for businesses)
energycost = 0.127
cost = energycost*W


# Cost per bed
cost_bed = cost/526

#Install cost
In=85000/365

#Lease

Lea=8000/365

#Insurance

Ins=9120/365

#Cost per day
Cost=cost+In+Lea+Ins
print('Total cost per day is $', Cost,'and the cost per bed per day is $', cost_bed)


Total cost per day is $ [281.365275908484] and the cost per bed per day is $ [0.00301226994425014]
